In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [9]:
# 3000x3000の座標平面を作成
x = np.linspace(0, 3000, 3000)
y = np.linspace(0, 3000, 3000)
X, Y = np.meshgrid(x, y)

# 電波発信端末の座標とGMMのパラメータを設定
transmitter_1 = [500, 500]  # 端末1の平均 (中心座標)
transmitter_2 = [2500, 2500]  # 端末2の平均 (中心座標)
transmitter_1 = [500, 2500]  # 端末3の平均 (中心座標)
transmitter_2 = [2500, 500]  # 端末4の平均 (中心座標)
covariance_matrix_1 = [[50000, 0], [0, 50000]]  # 端末1の共分散行列
covariance_matrix_2 = [[30000, 10000], [10000, 30000]]  # 端末2の共分散行列
covariance_matrix_1 = [[50000, 0], [0, 50000]]  # 端末3の共分散行列
covariance_matrix_2 = [[30000, 10000], [10000, 30000]]  # 端末4の共分散行列
weights = [0.5, 0.5]  # GMMの重み

# ガウス混合モデルを初期化
gmm = GaussianMixture(n_components=2, covariance_type="full", weights_init=weights)

# ガウス混合モデルを学習
gmm.means_init = np.array([transmitter_1, transmitter_2])
gmm.covariances_init = np.array([covariance_matrix_1, covariance_matrix_2])
gmm.fit(np.column_stack((X.ravel(), Y.ravel())))

# 各座標点での電波強度を計算
Z = -gmm.score_samples(np.column_stack((X.ravel(), Y.ravel())))
Z = Z.reshape(X.shape)

# Zの最大値を取得
Z_max = np.max(Z)

# 実際のRSSI値にスケーリング (例: -30dBm から -90dBm の範囲)
RSSI_max = -30  # dBm
RSSI_min = -90  # dBm

# 正規化された Z を実際のRSSI値にスケーリング
# 中心に近づくほどRSSIが大きく（0に近く）、外側に行くほど小さく（負の値が大きく）なるようにする
RSSI = RSSI_max + (Z / Z_max) * (RSSI_min - RSSI_max)

print(RSSI)

[[-89.61119711 -89.60049667 -89.58981053 ... -89.97687915 -89.98841314
  -89.99996145]
 [-89.60477908 -89.59407836 -89.58339194 ... -89.97020235 -89.98173606
  -89.99328409]
 [-89.59836542 -89.58766441 -89.57697771 ... -89.96352992 -89.97506335
  -89.9866111 ]
 ...
 [-89.98664973 -89.97510761 -89.96357981 ... -89.57684485 -89.58753641
  -89.59824228]
 [-89.99332268 -89.98178028 -89.9702522  ... -89.58325906 -89.59395033
  -89.60465593]
 [-90.         -89.98845732 -89.97692895 ... -89.58967764 -89.60036863
  -89.61107394]]


In [6]:
# 指定されたRSSI値の座標を絞り込む
target_RSSI = -60  # 絞り込みたいRSSI値 (例: -60dBm)
tolerance = 0.5  # 許容誤差 (例: ±0.5dBm)

# 絞り込み条件に合致する座標を取得
indices = np.where(
    (RSSI >= target_RSSI - tolerance) & (RSSI <= target_RSSI + tolerance)
)
filtered_coordinates = list(zip(X[indices], Y[indices]))

# 座標を丸める
rounded_coordinates = [(round(x), round(y)) for x, y in filtered_coordinates]

# 重複を削除
unique_coordinates = list(set(rounded_coordinates))

# プロット
plt.figure(figsize=(10, 8))
plt.cont    ourf(X, Y, RSSI, cmap="hot")
plt.colorbar(label="RSSI (dBm)")
plt.scatter(
    *zip(*unique_coordinates), color="blue", marker="x", label="Target RSSI Area"
)
plt.scatter(
    transmitter_1[0], transmitter_1[1], color="red", marker="o", label="Transmitter 1"
)
plt.scatter(
    transmitter_2[0], transmitter_2[1], color="green", marker="o", label="Transmitter 2"
)
plt.xlabel("X座標")
plt.ylabel("Y座標")
plt.title("電波強度のヒートマップ (RSSI)")
plt.legend()
plt.grid(True)
plt.show()

# 絞り込んだ座標の表示
print(f"指定されたRSSI値({target_RSSI}dBm)に近い座標数: {len(unique_coordinates)}")
if len(unique_coordinates) > 0:
    print("絞り込んだ座標のサンプル:")
    print(unique_coordinates[:10])  # 最初の10個を表示
else:
    print("指定されたRSSI値に近い座標が見つかりませんでした。")

SyntaxError: invalid syntax (pylabtools.py, line 170)